In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

import pandas as pd
import numpy as np
from copy import deepcopy

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time, auto_save_pickle

from models.fmin2 import fmin2, fmin2_model1
from models.nested_validation import *
from models.features import fixed_makedata, salvador_wrap, pretrained_wrap
from models.display import pickle2df

# models
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings("ignore")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


In [2]:
dir_save = os.path.normpath('../data/dic_20190819.pickle')
dic = load(dir_save)

amt_GI = load_pickle('../data/amt_hGI_pos.pickle')
for i, v in dic.items():
    if i in amt_GI.keys():
        v['AMT'] = amt_GI[i]
amt_ls = list(amt_GI.keys())
ls = amt_ls

exist
time: 14.6 s


In [3]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lgbm
    
    spaces = {'model1': model1, 'model2': LGBMClassifier,'method': method,
              'p2': p2_lgbm, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 20)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-23 17:01:36.647207
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 256, 'max_depth': 16, 'learning_rate': 0.15, 'gamma': 0.65, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 17:01:36.858009
[0.7904191616766467, 0.7924528301886792, 0.7741935483870969, 0.8421052631578948, 0.8441558441558441]
0.8086653295132322
-0.8086653295132322 -0.8086653295132322
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 512, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.55, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.75, 'bagging_fraction': 0.5, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 17:15:58.152860
[0.7999999999999999, 0.7950310559006211, 0.7870967741935484, 0.8488372093023256, 0.825]
0.811193007879299
-0.811193007879299 -0.811193007879299
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 64, 'max_depth': 8, 'learning_rat

,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.811,0.81,0.812,0.806,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.9, bagging_freq=10, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n importance_type='split', lambda_l2=50, learning_rate=0.1,\n max_depth=16, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=512,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=1, subsample_for_bin=200000,\n subsample_freq=0)","[nu, scale]",AMT,../pickle/2019-08-24 10:37:50.912344.pickle


save to ../csv/2019-08-24 10:37:51.778201.csv


,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.811,0.81,0.812,0.806,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.9, bagging_freq=10, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n importance_type='split', lambda_l2=50, learning_rate=0.1,\n max_depth=16, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=512,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=1, subsample_for_bin=200000,\n subsample_freq=0)","[nu, scale]",AMT,../pickle/2019-08-24 10:37:50.912344.pickle


time: 17h 36min 26s


### Combination1: NBBoW + nutritions

In [3]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lgbm
    
    spaces = {'model1': model1, 'model2': LGBMClassifier,'method': method,
              'p2': p2_lgbm, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 20)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['wc', 'nb', 'nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-22 11:27:29.876128
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 256, 'max_depth': 16, 'learning_rate': 0.15, 'gamma': 0.65, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-22 11:27:30.141912
[0.7831325301204819, 0.7926829268292682, 0.8205128205128205, 0.8284023668639053, 0.8518518518518519]
0.8153164992356656
-0.8153164992356656 -0.8153164992356656
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 512, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.55, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.75, 'bagging_fraction': 0.5, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-22 11:39:19.975216
[0.7757575757575756, 0.8271604938271605, 0.802547770700637, 0.8352941176470587, 0.8536585365853658]
0.8188836989035595
-0.8188836989035595 -0.8188836989035595
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 64, 'max_depth': 8

,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.831,0.823,0.84,0.825,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.75, bagging_freq=5, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.65,\n importance_type='split', lambda_l2=1, learning_rate=0.1,\n max_depth=4, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=64,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=0.7, subsample_for_bin=200000,\n subsample_freq=0)","[wc, nb, nu, scale]",AMT,../pickle/2019-08-22 22:58:46.230219.pickle


save to ../csv/2019-08-22 22:58:47.373844.csv


,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.831,0.823,0.84,0.825,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.75, bagging_freq=5, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.65,\n importance_type='split', lambda_l2=1, learning_rate=0.1,\n max_depth=4, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=64,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=0.7, subsample_for_bin=200000,\n subsample_freq=0)","[wc, nb, nu, scale]",AMT,../pickle/2019-08-22 22:58:46.230219.pickle


time: 11h 31min 29s


### glove pretrained and nutritions

In [3]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lgbm
    
    spaces = {'model1': model1, 'model2': LGBMClassifier,'method': method,
              'p2': p2_lgbm, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 20)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = pretrained_wrap(knowns, 'glove-wiki-gigaword-300')
for method in [['wv','nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-23 17:05:50.366943
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 256, 'max_depth': 16, 'learning_rate': 0.15, 'gamma': 0.65, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 17:05:50.648034
[0.8343558282208587, 0.8322981366459627, 0.802547770700637, 0.8674698795180723, 0.8917197452229298]
0.8456782720616921
-0.8456782720616921 -0.8456782720616921
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 512, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.55, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.75, 'bagging_fraction': 0.5, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 17:21:18.944176
[0.8263473053892216, 0.8383233532934131, 0.7792207792207793, 0.8520710059171598, 0.8875]
0.8366924887641147
-0.8366924887641147 -0.8456782720616921
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 64, 'max_depth': 8, 'learning_

,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.854,0.852,0.858,0.85,1,1,1,1,<models.features.pretrained_wrap object at 0x7efc5d4c97f0>,"LGBMClassifier(bagging_fraction=0.75, bagging_freq=5, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n importance_type='split', lambda_l2=3, learning_rate=0.1,\n max_depth=4, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=512,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=0.9, subsample_for_bin=200000,\n subsample_freq=0)","[wv, nu, scale]",AMT,../pickle/2019-08-24 09:17:20.483905.pickle


save to ../csv/2019-08-24 09:17:21.691140.csv


,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.854,0.852,0.858,0.85,1,1,1,1,<models.features.pretrained_wrap object at 0x7efc5d4c97f0>,"LGBMClassifier(bagging_fraction=0.75, bagging_freq=5, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n importance_type='split', lambda_l2=3, learning_rate=0.1,\n max_depth=4, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=512,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=0.9, subsample_for_bin=200000,\n subsample_freq=0)","[wv, nu, scale]",AMT,../pickle/2019-08-24 09:17:20.483905.pickle


time: 16h 17min 56s


### Combinations: skipthoughts + nutritons

In [5]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lgbm
    
    spaces = {'model1': model1, 'model2': LGBMClassifier,'method': method,
              'p2': p2_lgbm, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 20)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT', add_skip_thoughts='../data/X_skipthoughts_25k.pickle')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['skip','nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-23 16:43:39.816245
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 256, 'max_depth': 16, 'learning_rate': 0.15, 'gamma': 0.65, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 16:43:40.120437
[0.8220858895705522, 0.8242424242424242, 0.832214765100671, 0.8520710059171598, 0.8333333333333334]
0.832789483632828
-0.832789483632828 -0.832789483632828
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 512, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.55, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.75, 'bagging_fraction': 0.5, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 16:56:46.257884
[0.782608695652174, 0.8395061728395061, 0.8289473684210527, 0.8284023668639053, 0.8553459119496856]
0.8269621031452647
-0.8269621031452647 -0.832789483632828
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 64, 'max_depth': 8, 'le

,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.843,0.847,0.84,0.839,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.9, bagging_freq=20, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.5, gamma=0.45,\n importance_type='split', lambda_l2=0.1, learning_rate=0.1,\n max_depth=8, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=256,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=0.9, subsample_for_bin=200000,\n subsample_freq=0)","[skip, nu, scale]",AMT,../pickle/2019-08-24 14:23:54.371697.pickle


save to ../csv/2019-08-24 14:23:55.087730.csv


,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.843,0.847,0.84,0.839,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.9, bagging_freq=20, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.5, gamma=0.45,\n importance_type='split', lambda_l2=0.1, learning_rate=0.1,\n max_depth=8, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=256,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=0.9, subsample_for_bin=200000,\n subsample_freq=0)","[skip, nu, scale]",AMT,../pickle/2019-08-24 14:23:54.371697.pickle


time: 21h 40min 28s
